Explorar las diferentes librerías en Python para el procesamiento de audio y aplicar 5 de ellas para el tratamiento de un audio, el cual puede ser un sonido generado artificialmente, una canción, voz grabada, etc.
No limitarse a las técnicas exploradas en clase.

In [ ]:
import os
import librosa
import numpy as np
import noisereduce as nr
from pydub import AudioSegment
from pydub.silence import detect_nonsilent

Convierte un archivo de audio al formato WAV o exporta un AudioSegment existente.

Parámetros:
- input_file: str o AudioSegment
    Ruta al archivo de audio de entrada que se desea convertir, o un objeto AudioSegment.
- output_file: str
    Ruta donde se guardará el archivo de audio convertido en formato WAV.

In [ ]:
def convert_to_wav(input_file, output_file):
    if isinstance(input_file, str):
        audio = AudioSegment.from_file(input_file)
        print(f"Convertido a WAV desde archivo: {input_file} a {output_file}")
    elif isinstance(input_file, AudioSegment):
        audio = input_file
        print(f"Exportado AudioSegment a WAV: {output_file}")
    else:
        raise ValueError("input_file debe ser una ruta de archivo (str) o un objeto AudioSegment.")

    # Exportar el audio al formato WAV
    audio.export(output_file, format='wav')


Reduce el ruido de fondo en un archivo de audio.

Parámetros de entrada:
- audio: AudioSegment
    Objeto que representa el audio al que se le quiere reducir el ruido.

Salida:
- AudioSegment
    Objeto con el ruido reducido.

In [ ]:
def reduce_noise(audio):
    samples = np.array(audio.get_array_of_samples())
    reduced_noise = nr.reduce_noise(y=samples, sr=audio.frame_rate)
    reduced_audio = AudioSegment(
        reduced_noise.tobytes(),
        frame_rate=audio.frame_rate,
        sample_width=audio.sample_width,
        channels=audio.channels
    )
    print("Ruido reducido")
    return reduced_audio

Elimina el silencio al inicio y al final de un objeto de audio y devuelve el objeto de audio procesado.

Parámetros de entrada:
- audio: AudioSegment
    Objeto que representa el audio a procesar.
- silence_thresh: int, opcional
    Umbral de silencio en decibelios (dB). Por defecto es -50 dB.
- min_silence_len: int, opcional
    Longitud mínima de silencio en milisegundos para ser considerado como tal. Por defecto es 500 ms.

Salida:
- AudioSegment
    Objeto con el silencio recortado.

In [ ]:
def trim_silence(audio, silence_thresh=-40, min_silence_len=1000):
    non_silent_ranges = detect_nonsilent(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)
    if non_silent_ranges:
        start_trim = non_silent_ranges[0][0]
        end_trim = non_silent_ranges[-1][1]
        trimmed_audio = audio[start_trim:end_trim]
        print("Silencio recortado")
    else:
        trimmed_audio = audio
        print("No se detectó silencio significativo")
    return trimmed_audio

Elimina el silencio al inicio y al final de un objeto de audio y devuelve el objeto de audio procesado.

Parámetros de entrada:
- audio: AudioSegment
    Objeto que representa el audio a procesar.
- silence_thresh: int, opcional
    Umbral de silencio en decibelios (dB). Por defecto es 50 dB.

Salida:
- AudioSegment
    Objeto con el silencio recortado.

In [ ]:
def trim_silence_librosa(audio, silence_thresh=50):
    # Convertir AudioSegment a numpy array
    samples = np.array(audio.get_array_of_samples())
    sr = audio.frame_rate

    # Usar librosa para detectar silencios
    non_silent_intervals = librosa.effects.split(samples, top_db=silence_thresh)

    if len(non_silent_intervals) > 0:
        # Convertir índices de muestras a milisegundos
        start_trim = (non_silent_intervals[0][0] / sr) * 1000
        end_trim = (non_silent_intervals[-1][1] / sr) * 1000
        trimmed_audio = audio[start_trim:end_trim]
        print("Silencio recortado")
    else:
        trimmed_audio = audio
        print("No se detectó silencio significativo")

    return trimmed_audio

Normaliza el volumen de un objeto de audio al nivel objetivo especificado.

Parámetros de entrada:
- audio (AudioSegment): Objeto de audio que se desea normalizar.
- target_dBFS (float, opcional): Nivel de volumen objetivo en decibelios relativos al nivel de referencia (dBFS).

Salida:
- AudioSegment: Objeto de audio normalizado.

In [ ]:
def normalize_audio(audio, target_dBFS=-20.0):
    change_in_dBFS = target_dBFS - audio.dBFS
    normalized_audio = audio.apply_gain(change_in_dBFS)
    print("Audio normalizado")
    return normalized_audio

Aplica compresión dinámica a un objeto de audio.

Parámetros de entrada:
- audio (AudioSegment): Objeto de audio al que se desea aplicar compresión.
- threshold (float, opcional): Umbral en decibelios a partir del cual se aplica la compresión.
- ratio (float, opcional): Relación de compresión que determina cuánto se reduce el volumen por encima del umbral.
- attack (float, opcional): Tiempo en milisegundos que tarda la compresión en activarse.
- release (float, opcional): Tiempo en milisegundos que tarda la compresión en desactivarse.

Salida:
- AudioSegment
    Objeto con audio comprimido dinámicamente.

In [ ]:
def dynamic_compression(audio, threshold=-20.0, ratio=4.0, attack=5.0, release=50.0):
    compressed_audio = audio.compress_dynamic_range(
        threshold=threshold,
        ratio=ratio,
        attack=attack,
        release=release
    )
    print("Audio comprimido dinámicamente")
    return compressed_audio

Procesa todos los archivos de audio en un directorio y sus subdirectorios.
Aplica conversión a WAV, reducción de ruido, recorte de silencio, normalización y compresión dinámica.

Parámetros de entrada:
- root_directory: str
    Ruta al directorio raíz que contiene los archivos de audio a procesar.

No hay salida explícita, pero se generan archivos de audio procesados en el mismo directorio.

In [ ]:
def process_audio_files(root_directory):
    for current_folder, subfolders, files in os.walk(root_directory):
        for file in files:
            full_path = os.path.join(current_folder, file)
            name, extension = os.path.splitext(file)
            if extension.lower() not in ['.mp3', '.aac', '.flac', '.ogg', '.m4a']:
                continue  # Ignora archivos que no son de audio

            try:
                # Aplicar cada paso del procesamiento
                audio = AudioSegment.from_file(full_path)
                audio = reduce_noise(audio)
                audio = trim_silence(audio)
                audio = normalize_audio(audio)
                audio = dynamic_compression(audio)

                # Guardar el archivo final procesado y convertido
                output_file = os.path.join(root_directory,"convertido en wav", f"{name}_procesado.wav")
                convert_to_wav(audio, output_file)

            except Exception as e:
                print(f"Error al procesar {full_path}: {e}")


In [ ]:
root_directory = os.getcwd()  # Obtiene el directorio actual
process_audio_files(root_directory)

Ruido reducido
Silencio recortado
Audio normalizado
Audio comprimido dinámicamente
Exportado AudioSegment a WAV: c:\Users\Jesus\Downloads\Nueva carpeta\Grabación_procesado.wav
